In [4]:
import tensorflow as tf
from tensorflow.keras import models, layers


In [5]:
import matplotlib


In [6]:
import matplotlib.pyplot as plt
%matplotlib inline

In [7]:
import numpy as np

Loading the data into Tensorflow dataset

In [8]:
IMAGE_SIZE = 256
BATCH_SIZE = 16
RGB_CHANNELS = 3
EPOCHS = 50

In [9]:
dataset = tf_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    "pic_dataset", 
    shuffle=True,
    image_size=(IMAGE_SIZE,IMAGE_SIZE), 
    batch_size=BATCH_SIZE
)

Found 287 files belonging to 3 classes.


In [10]:
class_names = dataset.class_names
class_names

['blue_bin', 'orange_bin', 'purple_bun']

Spliting the data:
training -> 80%
validation -> 10%
test -> 10%

In [11]:
len(dataset)

18

In [ ]:
def get_partition_tf(ds, tarin_split=0.8, val_split=0.1, test_split=0.1, shuffle=True, shuffle_size = 100):
    train_size = tarin_split
    val_size = val_split
    test_size = test_split
    if shuffle:
        ds = ds.shuffle(shuffle_size, seed=8)
        
    train_dataset = ds.take(int(len(ds) * train_size))
    test_dataset = ds.skip(len(train_dataset))
    val_dataset = test_dataset.take(int(len(ds)*val_size))
    test_dataset = test_dataset.skip(len(val_dataset))
    return train_dataset, val_dataset, test_dataset
    

In [ ]:
train_dataset, val_dataset, test_dataset = get_partition_tf(dataset)

caching data to improve performance

In [ ]:
train_dataset = train_dataset.cache().shuffle(100).prefetch(buffer_size=tf.data.AUTOTUNE)
val_dataset = val_dataset.cache().shuffle(100).prefetch(buffer_size=tf.data.AUTOTUNE)
test_dataset = test_dataset.cache().shuffle(100).prefetch(buffer_size=tf.data.AUTOTUNE)

preprocessing the data. resize, rescale and use data augmentation

first, creating the layers for preprocessing

In [ ]:
resize_rescale_layer = tf.keras.Sequential([
    layers.experimental.preprocessing.Resizing(IMAGE_SIZE, IMAGE_SIZE),
    layers.experimental.preprocessing.Rescaling(1.0/255)
])

In [ ]:
data_augmentation_layer = tf.keras.Sequential([
    layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
    layers.experimental.preprocessing.RandomRotation(0.2)
])

second, we build the CNN (convolution neural network) model by connecting:
preprocessing layers -> data_augmentation_layer -> Cov layer -> polling layer... -> flaten data

In [ ]:
input_shape = (BATCH_SIZE, IMAGE_SIZE, IMAGE_SIZE, RGB_CHANNELS)
num_of_classes = 3
model = models.Sequential([
    resize_rescale_layer, 
    data_augmentation_layer,
    layers.Conv2D(16, (3,3), activation='relu', input_shape=input_shape),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(32, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(32, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(32, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(num_of_classes, activation='softmax')
    
])

model.build(input_shape=input_shape)

In [ ]:
model.summary()

In [ ]:
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

In [ ]:
history = model.fit(
    train_dataset, epochs=EPOCHS, batch_size=BATCH_SIZE, verbose=1, validation_data=val_dataset)

In [ ]:
scores = model.evaluate(test_dataset)


In [ ]:
scores

In [ ]:
history.history.keys()


In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

In [ ]:
for image_batch, labels_batch in test_dataset.take(1):
    img = image_batch[5].numpy().astype("int8")
    label = labels_batch[5].numpy()
    
    
    pred = model.predict(image_batch)
    print(f"real label is: {class_names[label]}, predict: {class_names[np.argmax(pred[5])]}")
    


In [ ]:
def predict(model, img):
    img_array = tf.keras.preprocessing.image.img_to_array(images[i].numpy())
    img_array = tf.expand_dims(img_array, 0)

    predictions = model.predict(img_array)

    predicted_class = class_names[np.argmax(predictions[0])]
    confidence = round(100 * (np.max(predictions[0])), 2)
    return predicted_class, confidence

In [ ]:
import os
model_ver = max([int(i) for i in os.listdir("../models")]) + 1
model_ver
model.save(f"../models/{model_ver}")